# 수집된 데이터

## 데이터 개요
- `x`, `y` 가 각각 위도, 경도입니다.
- 그 외에는 특이사항 없습니다.

## 데이터 수집 과정에서 특이사항
최대한 검색범위를 좁히는 게 중요합니다. 카카오맵 API 키워드 검색으로 획득할 수 있는 건 45*15 개의 데이터입니다. `동대문구 커피` 라고 검색하면 13,000 개 가량의 데이터가 카카오 지도 데이터베이스에 존재하는 걸 확인할 수 있지만, 최대 15개의 page 만을 불러올 수 있습니다. 그래서 [도로명이 모아져 있는 곳](https://www.code.go.kr/stdcode/roadCodeL.do)에서 동대문구의 도로명을 찾아다가 도로명 목록을 뽑고, `동대문구 ${도로명} 커피` 로 검색하였습니다.

In [ ]:
from google.colab import drive
from os import listdir

import pandas as pd

# basepath
dir_path = "/content/drive/MyDrive/2024-02-machine-learning/data"
file_name = "coffee_shops_in_ddm.csv"

# mount drive
drive.mount('/content/drive')

# read csv file
df = pd.read_csv(f"{dir_path}/{file_name}")

Mounted at /content/drive


In [ ]:
df

,address_name,category_group_code,category_group_name,category_name,id,phone,place_name,place_url,road_address_name,x,y
0,서울 동대문구 신설동 100-26,CE7,카페,음식점 > 카페,77517807,NaN,오슬랑커피,http://place.map.kakao.com/77517807,서울 동대문구 청계천로 443,127.026522,37.571278
1,서울 동대문구 용두동 255-58,CE7,카페,음식점 > 카페,473326435,02-929-6616,롤스커피 용두점,http://place.map.kakao.com/473326435,서울 동대문구 청계천로 483,127.030708,37.570255
2,서울 동대문구 용두동 255-48,CE7,카페,음식점 > 카페,13594216,070-4237-3865,카페이너프,http://place.map.kakao.com/13594216,서울 동대문구 청계천로 471,127.029390,37.570414
3,서울 동대문구 장안동 433-7,CE7,카페,음식점 > 카페 > 커피전문점 > 스타벅스,17884754,1522-3232,스타벅스 장한평역점,http://place.map.kakao.com/17884754,서울 동대문구 장한로 10,127.065416,37.562290
4,서울 동대문구 장안동 375-1,CE7,카페,음식점 > 카페 > 커피전문점 > 투썸플레이스,16620949,02-2212-1780,투썸플레이스 장안점,http://place.map.kakao.com/16620949,서울 동대문구 장한로 73,127.068377,37.567529
...,...,...,...,...,...,...,...,...,...,...,...
764,서울 동대문구 이문동 334-13,CE7,카페,음식점 > 카페,1689056369,123,신보,http://place.map.kakao.com/1689056369,서울 동대문구 회기로25길 101-13,127.057669,37.594599
765,서울 동대문구 회기동 45-100,CE7,카페,음식점 > 카페,180336939,0507-1348-3878,벤첸트,http://place.map.kakao.com/180336939,서울 동대문구 회기로21길 8-8,127.053387,37.591564
766,서울 동대문구 청량리동 205-758,CE7,카페,음식점 > 카페,822088399,02-966-9347,카페바움758,http://place.map.kakao.com/822088399,서울 동대문구 회기로 36,127.040594,37.591072
767,서울 동대문구 회기동 81,CE7,카페,음식점 > 카페 > 바나프레소,392460460,070-8998-8939,바나프레소 회기역사거리점,http://place.map.kakao.com/392460460,서울 동대문구 회기로 176,127.055481,37.590068


# 데이터 수집에 사용한 코드
- 확인을 위해 참고용으로 간단하게 남겨놓습니다.
- 로직만 참고해주세요.

In [ ]:
import requests
import time
import csv

# Kakao API Key 를 채워주세요
REST_API_KEY = ""

# 동대문구 행정동
distinct_list_of_dongdaemun = [
    "용신동",
    "제기동",
    "전농1동",
    "전농2동",
    "답십리1동",
    "답십리2동",
    "장안1동",
    "장안2동",
    "청량리동",
    "회기동",
    "휘경1동",
    "휘경2동",
    "이문1동",
    "이문2동",
]

# 동대문구 도로명 목록
road_list_of_dongdaemun = [
    "신이문로",
    "이문로",
    "회기로",
    "천호대로",
    "무학로",
    "답십리로",
    "왕산로",
    "보문로",
    "망우로",
    "고산자로",
    "외대역동로",
    "홍릉로",
    "약령시로",
    "약령중앙로",
    "화랑로",
    "사가정로",
    "장한로",
    "서울시립대로",
    "한빛로",
    "경동시장로",
    "제기로",
    "한천로",
    "청계천로",
    "천장산로",
    "황물로",
    "전농로",
    "휘경로",
    "안암로",
    "장안벚꽃로",
    "하정로",
    "난계로",
    "경희대로",
    "동부간선로",
    "겸재로",
    "정릉천동로",
    "종암로",
    "고미술로",
]


class KakaoLocalAPI:
    def __init__(self, api_key):
        self.__api_key = api_key
        self.__url_list = {
            "address": "https://dapi.kakao.com/v2/local/search/address",
            "keyword": "https://dapi.kakao.com/v2/local/search/keyword",
            "category": "https://dapi.kakao.com/v2/local/search/category",
        }
        self.__basic_header = {"Authorization": f"KakaoAK {self.__api_key}"}

    def get_location_info_by_keyword(self, keyword: str, page: int):
        params = {
            "query": keyword,
            "analyze_type": "similar",
            "page": page,
            "size": 15,
        }

        res = requests.get(
            url=self.__url_list["keyword"],
            params=params,
            headers=self.__basic_header,
        )

        return res.json()

    def write_to_csv(self, data, filename):
        # response 에 따라 headerfile 변경해주세요.
        header = [
            "address_name",
            "category_group_code",
            "category_group_name",
            "category_name",
            "distance",
            "id",
            "phone",
            "place_name",
            "place_url",
            "road_address_name",
            "x",
            "y",
        ]

        with open(filename, "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=header)
            writer.writeheader()
            for row in data:
                writer.writerow(row)


# create an instance of KakaoLocalAPI
instance = KakaoLocalAPI(REST_API_KEY)


def recursive_request_for_pagination(keyword, page=1):
    try:
        result = instance.get_location_info_by_keyword(keyword, page)
        instance.write_to_csv(
            data=result["documents"],
            filename=f"./data/dongdaemun_{keyword}_page_{page}.csv",
        )

        time.sleep(1)

        print(result["meta"])

        if result["meta"]["is_end"] is False:
            recursive_request_for_pagination(keyword, page + 1)

    except Exception as e:
        raise Exception(keyword, page, info=e)


for road in road_list_of_dongdaemun:
    try:
        recursive_request_for_pagination(f"동대문구 {road} 커피", 1)

    except Exception as e:
        print(e)
